# Table of Content

- [12.1 Starting and Stopping Threads](#12.1)
- [12.4 Locking Critical Sections](#12.4)
- [12.8 Performing Simple Parallel Progaming](#12.8)
- [12.9 Dealing with the GIL (and How to Stop Worring About it)](#12.9)

---
## <a name='12.1'></a> 12.1 Starting and Stopping Threads

In [1]:
import time
from threading import Thread

def countdown(n):
    while n > 0:
        print('T-minux', n)
        n -= 1
        time.sleep(5)
        
t = Thread(target=countdown, args=(10,))
t.start()

T-minux 10


In [2]:
if t.is_alive():
    print('Still Running')
else:
    print('Completed')

Still Running


In [3]:
t.join()

if t.is_alive():
    print('Still Running')
else:
    print('Completed')

T-minux 9
T-minux 8
T-minux 7
T-minux 6
T-minux 5
T-minux 4
T-minux 3
T-minux 2
T-minux 1
Completed


### Discussion

Due to Global Interpreter Lock(GIL), Python allows only one thread to exectute in the interpreter at any given time.

Thus, Python threads are suited for
- I/O handling
- Concurrent execution that performs block opertations(e.g. waiting for I/O, waiting result from database)  

and generally not be used for task that trying to achieve paralleism on multiple CPUs

---

## <a name='12.4'></a> 12.4 Locking Critical Sections

In [4]:
import threading

class SharedCounter:
    def __init__(self, initial_value=0):
        self._value = initial_value
        self._value_lock = threading.Lock()
        
    def incr(self, delta=1):
        with self._value_lock:
            self._value += delta
            
    def decr(self, delta=1):
        with self._value_lock:
            self._value -= delta

A `Lock` guarantees that only one thread is allowed to execute the block of statements under the `with` statement

---

## <a name='12.8'></a> 12.8 Performing Simple Parallel Progaming

**`concurrent.futures`** library provides a **`ProcessPoolExecutor`** class that can be used to execute computatinoally intensive functions

- Typical usage

```python
from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor() as pool:
    # Do work in parallel using pool
```

- Work must be submitted in the form of simple functions
    - instance methods, colusrues and others are not supported
- Arguments and return values must be compatible with **`pickle`**
- Functions submitted for work should not maintain persistent state or have side effects

---
## <a name='12.9'></a> 12.9 Dealing with the GIL (and How to Stop Worring About it)

Again, these techniques tend to only affect programs that are heavily CPU bound.  
For doing I/O, such as network communication, threads are often a sensible choice

### Method 1 - multiprocessing module

```python
pool = None

# Performs a large calculation (CPU bound)
def work(args):
    ...
    return result
    
# A thread that callas the above functin
def some_thread():
    while True:
        ...
        r = pool.apply(work, (args))
        ...
        
if __name__ == '__main__':
    import multiprocessing
    pool = multiprocessing.Pool()
```

The pool hands the work to a separate Python interpreter running in a different process.  
While the thread is waiting for the result, it releases the GIL

### Method 2 - C extension
Move computationally intensive tasks to C, and have the C code release the GIL.

If other tools are used to access C, such as the ctypes or Cython, you may not need to do anything.